In [1]:
import requests
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

### 1. Articles scrapping

In [21]:
with open('../.api_auth') as f:
    (api_login, api_pass) = [line.rstrip() for line in f]

ids = []
categories = []
leads = []
texts = []

date_range = pd.date_range(start="2022-01-01", end="2022-12-31").strftime('%d.%m.%Y').to_list()
for date in tqdm(date_range):
    r1 = requests.get(f'https://api.sta.si/news/sl/{date}', auth=(api_login, api_pass))
    articles_id = eval(r1.content)

    for id in articles_id:
        r2 = requests.get(f'https://api.sta.si/news/sta/{id}', auth=(api_login, api_pass))

        categories = r2.json().get('categories', [])
        if not any(category in ['SI', 'NA', 'PG', 'SU', 'SP'] for category in categories):
            ids.append(id)
            categories.append(categories)
            leads.append(r2.json().get('lede', ''))
            texts.append(r2.json().get('text', ''))
          
dataset = pd.DataFrame({'id': ids, 'categories': categories, 'lead': leads, 'text': texts})
dataset.to_json(f'../data/raw_articles/all_2022_2.jsonl', lines=True, orient='records')

100%|██████████| 365/365 [1:28:59<00:00, 14.63s/it]


ValueError: All arrays must be of the same length

### 2. Preparing evaluation dataset

In [12]:
# df = pd.read_json("..\\data\\raw_articles\\all_2023.jsonl", lines=True)
# df_eval = df.sample(100).reset_index(drop=True)
# df_eval.to_json(f'../data/raw_articles/all_2023_eval.jsonl', lines=True, orient='records', force_ascii=False)

In [48]:
# df = pd.read_json("..\\data\\raw_annotated\\sta_01.json", lines=True)['examples']

# id      = []
# content = []
# start   = []
# end     = []
# value   = []

# for i in range(len(df[0])):
#     row = df[0][i]
#     for annotation in row['annotations']:
#         if annotation['correct'] == True:
#             id.append(row['metadata']['id'])
#             content.append(row['content'])
#             start.append(annotation['start'])
#             end.append(annotation['end'])
#             value.append(annotation['value'])


In [67]:
# eval_df = pd.DataFrame({'id': id, 'text': content, 'entity': value, 'start': start, 'end': end})
# eval_df.to_json(f'../data/2023_eval.jsonl', lines=True, orient='records', force_ascii=False)

### 3. Ręczna anotacja zbioru testowego

In [9]:
# df = pd.read_json("..\\data\\raw_articles\\all_2022.jsonl", lines=True)

In [12]:
# df.sample(10000).reset_index(drop=True).to_json('../data/train_test/sampled_2022.jsonl', lines=True, orient='records', force_ascii=False)

In [16]:
# df = pd.read_json("..\\data\\train_test\\sampled_2022_ner.jsonl", lines=True)
# df.head()

,entity,context,article_id
0,SDS,SDS bo v prihodnjih dneh v DZ,3089645
1,DZ,SDS bo v prihodnjih dneh v DZ vložila zahtevo ...,3089645
2,DZ,"dolgotrajni oskrbi, kot jih je sprejel DZ .<b>...",3089645
3,ministrstvo za solidarno prihodnost,posameznih resorjev. Na novo bi oblikovali min...,3089645
4,ministrstvo za podnebje,ter za zdravje. Ustanovili bi tudi ministrstvo...,3089645


In [74]:
# labels = []
# test_sample = df.sample(100)

# from IPython.display import clear_output
# from google.cloud import translate_v2 as translate
# os.environ["PROJECT_ID"] = 'sound-memory-386318'

# i = 0
# for entity, text in zip(test_sample['entity'].to_list(), test_sample['context'].to_list()):
#     translate_client = translate.Client()
#     result = translate_client.translate(text, target_language="en")

#     print(f"========================={i}=======================")
#     print(f"Entity: {entity}")
#     print(f"Text: {result['input']}")
#     print(f"Translation: {result['translatedText']}")
#     print("==================================================")
#     labels.append(input())
#     clear_output()
#     i += 1